# Create Containerlab topology from CYJS graph

Initialize parameters   

In [25]:
topology_name = "nr-1"

Read CYJS graph data into a dictonary and initialize networkx graph from it

In [26]:
import json
import networkx as nx

cyjs = {}
with open(topology_name + ".cyjs", 'r', encoding='utf-8') as f:
    cyjs = json.load(f)
G = nx.cytoscape_graph(cyjs)
print(G.graph)

{'name': 'nr-1'}


Parse graph G into lists of: nodes and links. Keep a list of interfaces per device in `device_interfaces_map`.

In [27]:
nodes, links = [], []
device_interfaces_map = {}
for n in G.nodes:
    if G.nodes[n]['type'] == 'device':
        dev = G.nodes[n]['device']
        nodes.append(dev)
        device_interfaces_map[dev['name']] = {}
    elif G.nodes[n]['type'] == 'interface':
        int_name = G.nodes[n]['interface']['name']
        dev_name, dev_node_id = None, None
        peer_name, peer_dev_name, peer_dev_node_id = None, None, None
        for a_adj in G.adj[n].items():
            if G.nodes[a_adj[0]]['type'] == 'device':
                dev_name = G.nodes[a_adj[0]]['device']['name']
                dev_node_id = G.nodes[a_adj[0]]['device']['node_id']
                device_interfaces_map[dev_name][int_name] = ""
            elif G.nodes[a_adj[0]]['type'] == 'interface' and G.nodes[n]['side'] == 'a':
                peer_name = G.nodes[a_adj[0]]['interface']['name']
                for b_adj in G.adj[a_adj[0]].items():
                    if G.nodes[b_adj[0]]['type'] == 'device':
                        peer_dev_name = G.nodes[b_adj[0]]['device']['name']
                        peer_dev_node_id = G.nodes[b_adj[0]]['device']['node_id']
        if G.nodes[n]['side'] == 'a':
            links.append({
                'a': {
                    'node': dev_name,
                    'node_id': dev_node_id,
                    'interface': int_name,
                },
                'b': {
                    'node': peer_dev_name,
                    'node_id': peer_dev_node_id,
                    'interface': peer_name,
                },
            })

print(nodes)
print(device_interfaces_map)
print(links)

[{'id': 112, 'type': 'device', 'name': 'nr-rtr-01', 'node_id': 0}, {'id': 113, 'type': 'device', 'name': 'nr-sw-01', 'node_id': 3}, {'id': 114, 'type': 'device', 'name': 'nr-sw-02', 'node_id': 6}]
{'nr-rtr-01': {'Ethernet1/1': '', 'Ethernet1/2': ''}, 'nr-sw-01': {'Ethernet1/1': '', 'Ethernet1/2': ''}, 'nr-sw-02': {'Ethernet1/1': '', 'Ethernet1/2': ''}}
[{'a': {'node': 'nr-rtr-01', 'node_id': 0, 'interface': 'Ethernet1/1'}, 'b': {'node': 'nr-sw-01', 'node_id': 3, 'interface': 'Ethernet1/1'}}, {'a': {'node': 'nr-rtr-01', 'node_id': 0, 'interface': 'Ethernet1/2'}, 'b': {'node': 'nr-sw-02', 'node_id': 6, 'interface': 'Ethernet1/1'}}, {'a': {'node': 'nr-sw-01', 'node_id': 3, 'interface': 'Ethernet1/2'}, 'b': {'node': 'nr-sw-02', 'node_id': 6, 'interface': 'Ethernet1/2'}}]


Create container-compatible interface names for each device. We assume interface with index `0` is reserved for management, and start with `1`

In [28]:
for node, map in device_interfaces_map.items():
    # sort keys (interface names) in the map
    map_keys = list(map.keys())
    map_keys.sort()
    sorted_map = {k: f"eth{map_keys.index(k)+1}" for k in map_keys}
    device_interfaces_map[node] = sorted_map

print(device_interfaces_map)

for l in links:
    l['a']['c_interface'] = device_interfaces_map[l['a']['node']][l['a']['interface']]
    l['b']['c_interface'] = device_interfaces_map[l['b']['node']][l['b']['interface']]

print(links)

{'nr-rtr-01': {'Ethernet1/1': 'eth1', 'Ethernet1/2': 'eth2'}, 'nr-sw-01': {'Ethernet1/1': 'eth1', 'Ethernet1/2': 'eth2'}, 'nr-sw-02': {'Ethernet1/1': 'eth1', 'Ethernet1/2': 'eth2'}}
[{'a': {'node': 'nr-rtr-01', 'node_id': 0, 'interface': 'Ethernet1/1', 'c_interface': 'eth1'}, 'b': {'node': 'nr-sw-01', 'node_id': 3, 'interface': 'Ethernet1/1', 'c_interface': 'eth1'}}, {'a': {'node': 'nr-rtr-01', 'node_id': 0, 'interface': 'Ethernet1/2', 'c_interface': 'eth2'}, 'b': {'node': 'nr-sw-02', 'node_id': 6, 'interface': 'Ethernet1/1', 'c_interface': 'eth1'}}, {'a': {'node': 'nr-sw-01', 'node_id': 3, 'interface': 'Ethernet1/2', 'c_interface': 'eth2'}, 'b': {'node': 'nr-sw-02', 'node_id': 6, 'interface': 'Ethernet1/2', 'c_interface': 'eth2'}}]


Generate clab topology. Using this gist as inspiration https://gist.github.com/renatoalmeidaoliveira/fdb772a5a02f3cfc0b5fbe7e8b7586a2

In [29]:
topology = {
    'name': G.name,
    'nodes': [f"{n['name']}" for n in nodes],
    'links': [f"[\"{l['a']['node']}:{l['a']['c_interface']}\", \"{l['b']['node']}:{l['b']['c_interface']}\"]" for l in links],
}

print(json.dumps(topology, indent=4))

{
    "name": "nr-1",
    "nodes": [
        "nr-rtr-01",
        "nr-sw-01",
        "nr-sw-02"
    ],
    "links": [
        "[\"nr-rtr-01:eth1\", \"nr-sw-01:eth1\"]",
        "[\"nr-rtr-01:eth2\", \"nr-sw-02:eth1\"]",
        "[\"nr-sw-01:eth2\", \"nr-sw-02:eth2\"]"
    ]
}


Load Jinja2 template for Containerlab to run the topology through

In [30]:
from jinja2 import Environment, FileSystemLoader
env = Environment(
            loader=FileSystemLoader(f"."),
            line_statement_prefix='#'
        )
templ = env.get_template(f"clab.j2")

Run the topology through jinja2 template to get the final result

In [31]:
topo = templ.render(topology)
print(topo)
with open(topology_name + ".clab.yml", "w") as f:
    f.write(topo)

name: nr-1
topology:
    nodes:
        graphite:
            kind: linux
            image: netreplica/graphite:nanog86
            env:
                CLAB_SSH_CONNECTION: ${SSH_CONNECTION}
            binds:
                - __clabDir__/topology-data.json:/htdocs/lab/default/topology-data.json:ro
                - __clabDir__/ansible-inventory.yml:/htdocs/lab/default/ansible-inventory.yml:ro
            ports:
                - 8080:80
            exec:
                - sh -c 'graphite_motd.sh 8080'
            labels:
                graph-hide: yes

        nr-rtr-01:
            kind: ceos
            image: ceos
            binds:
                - nr-rtr-01_interface_map.json:/mnt/flash/EosIntfMapping.json:ro
        nr-sw-01:
            kind: ceos
            image: ceos
            binds:
                - nr-sw-01_interface_map.json:/mnt/flash/EosIntfMapping.json:ro
        nr-sw-02:
            kind: ceos
            image: ceos
            binds:
                - nr-s

Interface mapping file for cEOS

In [32]:
ceos_interfaces_templ = env.get_template(f"interface_maps/ceos.j2")
for d, m in device_interfaces_map.items():
    print(d)
    print(m)
    ceos_interface_map = ceos_interfaces_templ.render({'map': m})
    print(ceos_interface_map)
    with open(d + "_interface_map.json", "w") as f:
        f.write(ceos_interface_map)

nr-rtr-01
{'Ethernet1/1': 'eth1', 'Ethernet1/2': 'eth2'}
{
  "ManagementIntf": {
    "eth0": "Management0"
  },
  "EthernetIntf": {
    "eth1": "Ethernet1/1",
    "eth2": "Ethernet1/2"
  }
}
nr-sw-01
{'Ethernet1/1': 'eth1', 'Ethernet1/2': 'eth2'}
{
  "ManagementIntf": {
    "eth0": "Management0"
  },
  "EthernetIntf": {
    "eth1": "Ethernet1/1",
    "eth2": "Ethernet1/2"
  }
}
nr-sw-02
{'Ethernet1/1': 'eth1', 'Ethernet1/2': 'eth2'}
{
  "ManagementIntf": {
    "eth0": "Management0"
  },
  "EthernetIntf": {
    "eth1": "Ethernet1/1",
    "eth2": "Ethernet1/2"
  }
}
